# The imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
# import os
# import cv2
# import numpy as np
from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt

def process_image(image_path):
    """
    Processes an image using Daugman's algorithm-inspired techniques
    to demarcate pupil, iris, and sclera.
    """
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"Failed to load image: {image_path}")
        return

    # Step 1: Use Gaussian Blur to remove noise
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

    # Step 2: Apply adaptive threshold to highlight features
    _, threshold_image = cv2.threshold(blurred_image, 50, 255, cv2.THRESH_BINARY_INV)

    # Step 3: Find contours to locate the pupil
    contours, _ = cv2.findContours(threshold_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Step 4: Use the largest contour to find the pupil
    largest_contour = max(contours, key=cv2.contourArea)

    # Draw the pupil (approximating with a circle)
    (x, y), radius = cv2.minEnclosingCircle(largest_contour)
    center = (int(x), int(y))
    radius = int(radius)

    # Draw the pupil circle on the original image
    pupil_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.circle(pupil_image, center, radius, (0, 255, 0), 2)  # Draw green circle for pupil

    # Step 5: Use KMeans clustering to segment the iris and sclera
    reshaped_image = image.reshape((-1, 1))
    reshaped_image = np.float32(reshaped_image)

    # Define criteria and apply KMeans clustering
    kmeans = KMeans(n_clusters=3, random_state=0)
    labels = kmeans.fit_predict(reshaped_image)

    segmented_image = labels.reshape(image.shape)

    # The cluster with the smallest intensity corresponds to the pupil, others are iris and sclera
    pupil_cluster = np.argmin(kmeans.cluster_centers_)
    iris_cluster = np.median(kmeans.cluster_centers_)

    # Create a mask for pupil and iris
    pupil_mask = (segmented_image == pupil_cluster).astype(np.uint8) * 255
    iris_mask = (segmented_image == iris_cluster).astype(np.uint8) * 255

    # Overlay the masks on the original image for demarcation
    segmented_result = np.zeros_like(pupil_image)
    segmented_result[pupil_mask == 255] = [0, 255, 0]   # Green for pupil
    segmented_result[iris_mask == 255] = [255, 0, 0]    # Blue for iris

    # Combine all the masks and display the sclera (in white)
    sclera_mask = (segmented_image != pupil_cluster) & (segmented_image != iris_cluster)
    segmented_result[sclera_mask] = [255, 255, 255]     # White for sclera

    # Step 6: Show the result or save it
    combined = cv2.addWeighted(pupil_image, 0.7, segmented_result, 0.3, 0)
    plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
    plt.title("Pupil, Iris, and Sclera Demarcation")
    plt.axis('off')
    plt.show()

def process_images_in_directory(directory, image_list):
    """
    Walks through the directory and processes specified images.
    """
    for root, _, files in os.walk(directory):
        for file in files:
            if file in image_list:
                image_path = os.path.join(root, file)
                print(f"Processing {image_path}...")
                process_image(image_path)


process_images_in_directory(directory, image_list)


/bin/bash: line 1: htop: command not found
